<a href="https://colab.research.google.com/github/hihohub/umbrella-networks/blob/main/custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Umbrella networks
- copyright 2021 by James Smith
- contact - derekjsmit@gmail.com

Demonstration instructions
- How to load custom files into an umbrella network and pass custom model to the network.
- Organize your image files into folders by category. Multiple levels are best. For example, (book covers) -> (scifi), (fantasy), (mystery).
- First, run the large cell containing the umbrella code, then scroll down 1333 lines to the experiment.
- Change the folder name in the demonstration code to your folder path.
- The rest of the code should work as written.

Umbrella network instructions<br/>
- The umbrella code was designed for reading a local folder and automatically constructing the hierarchical network from the implicit hierarchy in the subfolders.
- If have images in folders, initialize with images_folder = path. Umbrella network constructs tree by itself. Refer to examples.
- However, it also has functions for reading the data from an online repository and building the tree node by node (since the data has no folder structure).
- If loading data from repository, initialize with images_folder = None, then use repo functions. Must first preprocess the data (normalize, reshape). Only process x data, not labels. Umbrella network constructs labels by itself. However, subdivide classes by label, then manually construct tree with repo functions. Finally, init_from_repo(). Refer to examples.
- In repo mode, umbrella network throws away labels and makes its own. For example, if divide MNIST into odd and even numbers, each subset will have labels 0-4. Hence, anything other than flat topology requires that the umbrella network form new labels.






In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


%%%%%%%%%%%%%%%%%%<br/>
UMBRELLA NETWORK<br/>
%%%%%%%%%%%%%%%%%%<br/>

In [ ]:
! wget https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py

--2022-04-15 04:12:57--  https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48219 (47K) [text/plain]
Saving to: ‘umbrella.py’

umbrella.py         100%[===================>]  47.09K  --.-KB/s    in 0.01s   

2022-04-15 04:12:57 (4.57 MB/s) - ‘umbrella.py’ saved [48219/48219]



In [ ]:
from umbrella import Umbrella_Network,Umbrella_Label,Umbrella_Node,Joint_Probability_Network

In [ ]:
import os
import sys
import numpy as np
import math
import random
from PIL import Image, ImageOps, ImageChops
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, add, BatchNormalization, Dropout, AveragePooling2D, GlobalAveragePooling2D, SeparableConv2D
from tensorflow.keras.utils import to_categorical
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras import backend as backend
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pickle
import copy
import time

- ##################
- Demonstrate loading images files into umbrella networks and passing a custom model to the network
- ##################

FLAT MODEL (standard softmax over every class)

In [ ]:
network_type = 'softmax'
images_folder = "/content/drive/My Drive/5390/project/small_hierarchy"
serial_file = "/content/drive/My Drive/Colab Notebooks/small_hierarchy.dat"
models_folder = "/content/drive/My Drive/Colab Notebooks/models"
target_image_size = 150
levels = 0 # if levels = 0, reads just last level folders, makes flat softmax over all classes
channels = 3
valid_ratio = 0.2 # validation set
test_ratio = 0.0 # test set
initialize_now = True
SOFTMAX = Umbrella_Network(network_type,images_folder,serial_file,models_folder,target_image_size,channels,levels,valid_ratio,test_ratio,initialize_now)

In [ ]:
SOFTMAX.traverse_training_data()

small_hierarchy trainX 137 (137,) trainY 137 (137, 10)
   alligator trainX 0 (0,) trainY 0 (0,)
   mammals trainX 0 (0,) trainY 0 (0,)
   salamanders trainX 0 (0,) trainY 0 (0,)
   snake trainX 0 (0,) trainY 0 (0,)
   automobiles trainX 0 (0,) trainY 0 (0,)
   bicycles trainX 0 (0,) trainY 0 (0,)
   boats trainX 0 (0,) trainY 0 (0,)
   cactus trainX 0 (0,) trainY 0 (0,)
   hedges trainX 0 (0,) trainY 0 (0,)
   shrubs trainX 0 (0,) trainY 0 (0,)


In [ ]:
# custom model rather than default
def factory(node):
    node.model = Sequential()
    node.model.add(Conv2D(16,(3,3),activation='relu',input_shape=node.trainX[0].data.shape))
    node.model.add(Conv2D(16,(3,3),activation='relu',input_shape=node.trainX[0].data.shape))
    node.model.add(Conv2D(32,(3,3),activation='relu'))
    node.model.add(Conv2D(32,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(64,(3,3),activation='relu'))
    node.model.add(Conv2D(64,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(128,(3,3),activation='relu'))
    node.model.add(Conv2D(128,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(256,(3,3),activation='relu'))
    node.model.add(Conv2D(256,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(512,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Flatten())
    node.model.add(Dense(32,activation='relu'))
    node.model.add(Dense(16,activation='relu'))
    node.model.add(Dense(len(node.children),activation='softmax'))
    node.model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
verbose = True
SOFTMAX.make_models(factory,verbose)

making model for small_hierarchy
small_hierarchy
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 conv2d_9 (Conv2D)           (None, 146, 146, 16)      2320      
                                                                 
 conv2d_10 (Conv2D)          (None, 144, 144, 32)      4640      
                                                                 
 conv2d_11 (Conv2D)          (None, 142, 142, 32)      9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 71, 71, 32)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 69, 69, 64)        18496     
     

In [ ]:
SOFTMAX.train_models(eps=1,batch=64,aug=True,vrb=True)

training model for small_hierarchy
trainX 137
trainY 137
3/3 [==============================] - 14s 543ms/step - loss: 2.3013 - accuracy: 0.0511


In [ ]:
SOFTMAX.get_training_accuracy()
SOFTMAX.predict_validation_set() # set verbose=True to view all the images
SOFTMAX.predict_validation_top5()

average training accuracy 0.051095
average training loss 2.301316
34

average validation accuracy 0.117647
top 5 accuracy 0.59


UMBRELLA NETWORK WITH TWO LEVELS

In [ ]:
network_type = 'softmax'
images_folder = "/content/drive/My Drive/5390/project/small_hierarchy"
serial_file = "/content/drive/My Drive/Colab Notebooks/small_hierarchy.dat"
models_folder = "/content/drive/My Drive/Colab Notebooks/models/models"
target_image_size = 150
channels = 3
levels = 2 # the folders actually have four or five levels, but setting levels = 2 will compress the lower folders into one level
valid_ratio = 0.2
test_ratio = 0.0
initialize_now = True
SOFTMAX_UMBRELLA = Umbrella_Network(network_type,images_folder,serial_file,models_folder,target_image_size,channels,levels,valid_ratio,test_ratio,initialize_now)

warning - could not find models folder


In [ ]:
def factory(node):
    node.model = Sequential()
    node.model.add(Conv2D(16,(3,3),activation='relu',input_shape=node.trainX[0].data.shape))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(32,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(64,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(128,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Flatten())
    node.model.add(Dense(32,activation='relu'))
    node.model.add(Dense(len(node.children),activation='softmax'))
    node.model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
SOFTMAX_UMBRELLA.make_models(factory=factory,verbose=True)

making model for small_hierarchy
small_hierarchy
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_20 (Conv2D)          (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_21 (Conv2D)          (None, 34, 34, 64)        18496     
                                                                 
 max

In [ ]:
SOFTMAX_UMBRELLA.train_models(eps=1,batch=10,aug=False,vrb=True)

training model for small_hierarchy
trainX 137
trainY 137
14/14 [==============================] - 1s 21ms/step - loss: 1.1257 - accuracy: 0.3723
training model for animals
trainX 35
trainY 35
4/4 [==============================] - 1s 56ms/step - loss: 1.3778 - accuracy: 0.3143
training model for objects
trainX 51
trainY 51
6/6 [==============================] - 1s 41ms/step - loss: 1.1015 - accuracy: 0.3922
training model for plants
trainX 51
trainY 51
6/6 [==============================] - 1s 7ms/step - loss: 1.0957 - accuracy: 0.3922


In [ ]:
SOFTMAX_UMBRELLA.get_training_accuracy()
SOFTMAX_UMBRELLA.predict_validation_set()
SOFTMAX_UMBRELLA.predict_validation_top5()

average training accuracy 0.367716
average training loss 1.175176
34

average validation accuracy 0.147059
top 5 accuracy 0.65
